### Meta Labeling

베팅(매도 또는 매수)의 방향을 설정하는 모델을 이미 갖고 있는 상황을 가정해 보자. 이 경우에는 베팅의 크기만 알면 되는데, 베팅이 없을 가능성도 있다. 이는 현업에서 항상 겪는 상황이기도 하다. 우리는 대개 매수할 것인지 매도할 것인지를 알고 있으므로, 남은 문제는 얼마를 집행할 것인지를 결정하는 것이다. 단지 머신러닝 알고리즘이 매수 매도 등 베팅의 방향을 학습한 후 적절한 베팅 크기가 무엇인지 쉽게 알려 주기를 원하지 않는다. 아마도 이 문제에 관한 어떠한 책이나 논문이 나오지 않았다는 것에 이미 익숙해져 버렸을 수 있다

다행히 이 문제를 여기서 해결해 볼 수 있다. Lopez de Prado는 이 문제를 Meta Labeling이라 부르는데 그 이유는 1차 외생 모델을 어떻게 사용할지 학습하는 부수적인 머신러닝 모델을 구축하고자 하기 때문이다

아래의 과정은 Triple Barrier Method에서 구축한 방법과 동일한 과정이다.

In [19]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from FinancialMachineLearning.filter.filter import cusum_filter
from FinancialMachineLearning.labeling.labeling import *
from FinancialMachineLearning.features.volatility import daily_volatility

start_date = '2000-01-01'
end_date = '2024-04-06'
ticker = 'AAPL'
data = yf.download(
    ticker, 
    start = start_date, 
    end = end_date
)

[*********************100%%**********************]  1 of 1 completed


In [2]:
cusum_events = cusum_filter(
    data['Close'], 
    threshold = data['Close'].diff().mean() * 3, 
    time_stamps = False
)

cusum_events = pd.DatetimeIndex(cusum_events)

In [3]:
vertical_barrier = add_vertical_barrier(
    cusum_events, 
    data['Close'], 
    num_days = 7 # expariation limit
)
vertical_barrier.head()

2000-01-04   2000-01-11
2000-01-11   2000-01-18
2000-01-13   2000-01-20
2000-01-19   2000-01-26
2000-01-28   2000-02-04
Name: Date, dtype: datetime64[ns]

In [4]:
volatility = daily_volatility(
    data['Close'], 
    lookback = 60 # moving average span
)

In [5]:
triple_barrier_events = get_events(
    close = data['Close'],
    t_events = cusum_events[1:],
    pt_sl = [2, 1], # profit taking 2, stopping loss 1
    target = volatility, # dynamic threshold
    min_ret = 0.01, # minimum position return
    num_threads = 1, # number of multi-thread 
    vertical_barrier_times = vertical_barrier, # add vertical barrier
    side_prediction = None # betting side prediction (primary model)
)
triple_barrier_events.head()

,t1,trgt,pt,sl
2000-01-11,2000-01-12,0.024043,2,1
2000-01-13,2000-01-19,0.050703,2,1
2000-01-19,2000-01-26,0.081573,2,1
2000-01-28,2000-02-04,0.069135,2,1
2000-02-07,2000-02-14,0.063289,2,1


In [6]:
labels = meta_labeling(
    triple_barrier_events, 
    data['Close']
)

In [8]:
labels.head()

,ret,trgt,bin
2000-01-11,-0.059973,0.024043,-1
2000-01-13,0.101422,0.050703,0
2000-01-19,0.034017,0.081573,0
2000-01-28,0.062731,0.069135,0
2000-02-07,0.015342,0.063289,0


In [17]:
triple_barrier_events['side'] = labels['bin']
triple_barrier_events.head()

,t1,trgt,pt,sl,side
2000-01-11,2000-01-12,0.024043,2,1,-1.0
2000-01-13,2000-01-19,0.050703,2,1,0.0
2000-01-19,2000-01-26,0.081573,2,1,0.0
2000-01-28,2000-02-04,0.069135,2,1,0.0
2000-02-07,2000-02-14,0.063289,2,1,0.0


In [18]:
meta_labels = meta_labeling(
    triple_barrier_events, # with side labels
    data['Close']
)
meta_labels.head()

,ret,trgt,bin,side
2000-01-11,0.063799,0.024043,1,-1.0
2000-01-13,0.000000,0.050703,0,0.0
2000-01-19,0.000000,0.081573,0,0.0
2000-01-28,0.000000,0.069135,0,0.0
2000-02-07,0.000000,0.063289,0,0.0


여기서 생성된 `meta_labels['bin']`이 가능한 값은 {-1, 0, 1}과 반대로 {0, 1}이다. Machine Learning 알고리즘은 오직 베팅을 하느냐, 하지 않느냐만 결정하고자 학습된다. 예측 label이 1이라면 2차 모델의 확률을 사용해 베팅의 크기를 알아내는데 포지션의 side(매수 혹은 매도인지)는 이미 primary model에 의해 결정되어 있다

### How to use Meta Labeling